In [18]:
"""
Build an association set from the EHR for a specific set of drugs and a specific set of conditions.

Uses a window of time between condition and drug in order for the event to count.
"""
import os
import sys
import csv
import argparse

import tqdm
import MySQLdb
import MySQLdb.cursors

from itertools import combinations

from collections import defaultdict

from datetime import date
from datetime import timedelta

In [19]:
# TODO: make the params available from the command line
# all this is commented out b/c I haven't bothered figuring out parser
"""
parser = argparse.ArgumentParser(description = 'Queries the database to build a patient timeline and creates a file suitable for an association statistic study.')
parser.add_argument('--exposure-drugs', help =
  'SNOWMED concept codes used to define drug exposure')
parser.add_argument('--case-conditions', help = 
  'SNOWMED concept codes used to define case conditions')
parser.add_argument('--control-conditions', default = 'faers_conditions.txt', help = 
  'SNOWMED concept codes used to define control conditions; case codes are automatically removed')
parser.add_argument('--condition-window', default = 7L, type = int, help =
  'the number of scan-forward days from a condition to cluster together into a single event')
parser.add_argument('--control-window', default = 30L, type = int, help =
  'the number of scan-backward days to the most-recent drug to define a control event')
parser.add_argument('--case-window', default = 30L, type = int, help =
  'the number of scan-backward days to the most-recent drug to define a case event')
parser.add_argument('--drug-expiry', default = 30L, type = int, help = 
  'the number of scan-backward days to define the current drug set')
parser.add_argument('--first-condition-only', default = True, type = bool, help = 
  'whether or not to restrict conditions to the fist observed instance')
parser.add_argument('--first-exposure-only', default = True, type = bool, help = 
  'whether or not to restrict exposures to the fist observed instance')
parser.add_argument('--print-every', default = 100000, type = int, help = 
  'when querying the database, print an update every n rows processed')
"""



"\nparser = argparse.ArgumentParser(description = 'Queries the database to build a patient timeline and creates a file suitable for an association statistic study.')\nparser.add_argument('--exposure-drugs', help =\n  'SNOWMED concept codes used to define drug exposure')\nparser.add_argument('--case-conditions', help = \n  'SNOWMED concept codes used to define case conditions')\nparser.add_argument('--control-conditions', default = 'faers_conditions.txt', help = \n  'SNOWMED concept codes used to define control conditions; case codes are automatically removed')\nparser.add_argument('--condition-window', default = 7L, type = int, help =\n  'the number of scan-forward days from a condition to cluster together into a single event')\nparser.add_argument('--control-window', default = 30L, type = int, help =\n  'the number of scan-backward days to the most-recent drug to define a control event')\nparser.add_argument('--case-window', default = 30L, type = int, help =\n  'the number of scan-bac

In [31]:
drugs_file_exposure     = 'arrhythmia_drugs.txt'
drugs_file_non_exposure = 'faers_drugs.txt'
conditions_file_case    = 'arrhythmia_conditions.txt'
conditions_file_control = 'faers_conditions.txt'



In [22]:
exposure_window_case = 30
exposure_window_control = 30
keep_only_first_exposure = False
keep_only_first_condition = True
num_controls_per_patient = 100000
num_cases_per_patient = 100000
self_control_style = "past" # past, all, none
print_every = 1000000


In [23]:
# output is assocation statistics
output_file_name = "assoc_" + str(exposure_window_case) + "_" + str(exposure_window_control) + ".csv"
# counts is just how many patients have that number of events
counts_file_name = "counts_" + str(exposure_window_case) + "_" + str(exposure_window_control) + ".csv" 


In [24]:
counts_file_name

'counts_30_30.csv'

In [27]:
cd '/Users/owner/Downloads/faers_demo/assoc'

/Users/owner/Downloads/faers_demo/assoc


In [132]:
drugs_exposure     = set(int(line.strip()) for line in open(drugs_file_exposure, 'r'))
drugs_non_exposure = set(int(line.strip()) for line in open(drugs_file_non_exposure, 'r'))
conditions_case    = set(int(line.strip()) for line in open(conditions_file_case, 'r'))
conditions_control = set(int(line.strip()) for line in open(conditions_file_control, 'r'))

In [146]:
drugs_exposure

{1307542,
 1309944,
 1335606,
 1344996,
 1351447,
 1351461,
 1353256,
 1354860,
 1360421,
 1362979,
 19006969,
 19038998,
 19045581,
 19050087,
 19051513,
 19069075,
 19105879,
 19135792,
 40163615}

In [134]:
len(drugs_non_exposure)

4245

In [144]:
drugs_non_exposure = drugs_non_exposure - drugs_exposure
conditions_control = conditions_control - conditions_case


In [145]:
drugs_exposure

{1307542,
 1309944,
 1335606,
 1344996,
 1351447,
 1351461,
 1353256,
 1354860,
 1360421,
 1362979,
 19006969,
 19038998,
 19045581,
 19050087,
 19051513,
 19069075,
 19105879,
 19135792,
 40163615}

In [136]:
len(drugs_non_exposure)

4227

In [117]:
con = MySQLdb.connect(host='127.0.0.1', port=3307, user='ss5987', passwd='PNvb8xa6f3', cursorclass = MySQLdb.cursors.SSCursor)
cur = con.cursor()

In [118]:
cur.execute("use clinical_merge_v5_260318")



0

In [78]:
# need to map from the very specific drugs that appear in the drug_exposure table to 
# their ingredients, which is done by finding their ancestors


query = """
SELECT ca.descendant_concept_id, ca.ancestor_concept_id
  FROM concept_ancestor ca
  JOIN concept c ON c.concept_id = ca.ancestor_concept_id
 WHERE c.concept_class_id = 'Ingredient' limit 0,50000
"""

drug2ingredient = defaultdict(dict)

__ = cur.execute(query)
for row in cur:
    descendant_id, ancestor_id = row
    
    if descendant_id not in drug2ingredient:
        drug2ingredient[descendant_id] = set()
    drug2ingredient[descendant_id].add(ancestor_id)



In [169]:
drug2ingredient

defaultdict(dict,
            {35786784: {501343},
             36884884: {501343},
             46275211: {501343},
             41263633: {501343},
             46275254: {501343},
             19133078: {501343},
             41406356: {501343},
             43687828: {501343},
             43624791: {501343},
             36409816: {501343},
             36405043: {501343},
             41357704: {501343},
             501345: {501343},
             36405739: {501343},
             41390064: {501343},
             36404272: {501343},
             41458358: {501343},
             21088054: {501343},
             43627040: {501343},
             46275215: {501343},
             40866861: {501343},
             43680746: {501343},
             44106142: {501343},
             41268994: {501343},
             21096317: {501343},
             21174909: {501343},
             44082860: {501343},
             41390781: {501343},
             40155316: {501343},
             21137396: {501

In [83]:
__ = cur.execute("select count(*) from drug_exposure")
num_rows = cur.fetchall()[0][0]
num_rows

78949941

In [91]:
query = """
SELECT person_id, drug_concept_id, drug_exposure_start_date
  FROM drug_exposure limit 0,50000
"""

person2drug2date = defaultdict(dict)

print ("Fetching drug exposure data")
__ = cur.execute(query)
row_num = 1
for row in cur:
    person_id, drug_concept_id, drug_exposure_start_date = row
    
    if drug_concept_id == 0:
        if row_num % print_every == 0:
            print ("processed %d of %d rows" % ( row_num, num_rows ))
        row_num += 1
        continue
    
    if drug_concept_id not in drug2ingredient:
        drug2ingredient[drug_concept_id] = set()
        drug2ingredient[drug_concept_id].add(drug_concept_id)
    
    for ingredient_id in drug2ingredient[drug_concept_id]:
        if ingredient_id in drugs_exposure or ingredient_id in drugs_non_exposure:
            if keep_only_first_exposure:
                if not ingredient_id in person2drug2date[person_id] or \
                   drug_exposure_start_date < person2drug2date[person_id][ingredient_id]:
                    person2drug2date[person_id][ingredient_id] = drug_exposure_start_date
            else:
                if not ingredient_id in person2drug2date[person_id]:
                    person2drug2date[person_id][ingredient_id] = set()
                person2drug2date[person_id][ingredient_id].add(drug_exposure_start_date)
    
    if row_num % print_every == 0:
        print (" processed %d of %d rows" % ( row_num, num_rows ))
    row_num += 1

print ("Completed loading drug exposure data for %d patients." % len(person2drug2date))


Fetching drug exposure data
Completed loading drug exposure data for 143 patients.


In [128]:
person2drug2date

defaultdict(dict,
            {9: {42903913: {datetime.date(2011, 4, 26),
               datetime.date(2011, 10, 10),
               datetime.date(2012, 5, 15),
               datetime.date(2013, 9, 3)}},
             29: {705103: {datetime.date(2013, 7, 24),
               datetime.date(2013, 7, 26)}},
             31: {705103: {datetime.date(2011, 7, 20),
               datetime.date(2012, 1, 31),
               datetime.date(2012, 2, 1)}},
             92: {705944: {datetime.date(2013, 10, 16),
               datetime.date(2013, 10, 22),
               datetime.date(2013, 11, 5),
               datetime.date(2013, 11, 22),
               datetime.date(2013, 12, 2)},
              703547: {datetime.date(2008, 7, 3)}},
             97: {710062: {datetime.date(2010, 1, 23)}},
             118: {705103: {datetime.date(2007, 10, 22),
               datetime.date(2007, 10, 24)}},
             124: {705103: {datetime.date(2003, 11, 8),
               datetime.date(2003, 11, 10)},
         

In [112]:
query = """
SELECT condition_concept_id, icd_chapter
  FROM cumc_condition_concept_to_icd_chapter limit 0,50000
"""
cur.execute(query)
code2chapter = defaultdict(dict)
for condition_concept_id, icd_chapter in cur.fetchall():
    code2chapter[condition_concept_id] = icd_chapter



In [113]:
__ = cur.execute("select count(*) from condition_occurrence")
num_rows = cur.fetchall()[0][0]
num_rows

140300457

In [119]:
query = """
SELECT person_id, condition_concept_id, condition_start_date
  FROM condition_occurrence limit 0,50000
"""

person2condition2date = defaultdict(dict)
case_patients = set() # track cases in case we have special self-control rules

print ("Fetching condition data")
__ = cur.execute(query)
row_num = 1
for row in cur:
    person_id, condition_concept_id, condition_start_date = row
    
    if condition_concept_id == 0:
        if row_num % print_every == 0:
            print( " processed %d of %d rows" % ( row_num, num_rows ))
        row_num += 1
        continue
    
    if condition_concept_id in conditions_case or condition_concept_id in conditions_control:
        if keep_only_first_condition:
            if not condition_concept_id in person2condition2date[person_id] or \
               condition_start_date < person2condition2date[person_id][condition_concept_id]:
                person2condition2date[person_id][condition_concept_id] = condition_start_date
        else:
            if not condition_concept_id in person2condition2date[person_id]:
                person2condition2date[person_id][condition_concept_id] = set()
            person2condition2date[person_id][condition_concept_id].add(condition_start_date)
        
        if condition_concept_id in conditions_case:
            case_patients.add(person_id)
    
    if row_num % print_every == 0:
       print ("  processed %d of %d rows" % (row_num, num_rows))
    row_num += 1


print ("Completed loading condition data for %d patients." % len(person2condition2date))

Fetching condition data
Completed loading condition data for 597 patients.


In [167]:
person2condition2date[2][80767]

datetime.date(2015, 12, 3)

In [96]:
__ = cur.execute("select count(*) from person")
num_rows = cur.fetchall()[0][0]
num_rows

6377222

In [98]:
query= """
SELECT person_id, year_of_birth, gender_concept_id
  FROM person limit 0,50000
"""

person2demog = dict()
print ("Fetching demographics data")
__ = cur.execute(query)
row_num = 1
for row in cur:
    person_id, yob, gender = row
    person2demog[person_id] = { 'yob': yob, 'gender': gender }
    
    if row_num % print_every == 0:
       print ("  processed %d of %d rows" % ( row_num, num_rows ))
    row_num += 1

print ("Completed loading demographic data for %d patients." % len(person2demog)
)

Fetching demographics data
Completed loading demographic data for 50000 patients.


In [99]:
# maps and sets accumulated so far:
#   person2drug2date - map of person id to first date prescribed for drug id
#   person2condition2date - map of person id to the first date receiving condition
#   person2demog - map of person id to demo tuple
#   case_patients - set containing those person_ids corresponding to cases

# relevant parameters:
#   exposure_window_case
#   exposure_window_control
#   num_controls_per_patient
#   num_cases_per_patient
#   self_control_style = "past" # past, all, none


In [100]:
min_drug_date = date(2001, 1, 1) # throw out everything before

def sort_and_str(s):
    """ converts a set into a sorted string """
    return ';'.join(map(str, sorted(s)))


In [159]:
def get_drugs(person_id):
    """ returns a sorted array of drug exposures """
    drugs = []
    
    if keep_only_first_exposure:
        for drug_id, drug_date in person2drug2date[person_id].items():
            if drug_date is not None: drugs.append({ 'id' : drug_id, 'date' : drug_date })
    else:
        for drug_id, drug_dates in person2drug2date[person_id].items():
            for drug_date in drug_dates:
                if drug_date is not None: drugs.append({ 'id' : drug_id, 'date' : drug_date })
    
    drugs.sort(key = lambda drug: drug['date'])
    return drugs


In [122]:
def get_conditions(person_id):
    """ returns a sorted array of conditions """
    conditions = []
    
    if keep_only_first_condition:
        for condition_id, condition_date in person2condition2date[person_id].items():
            if condition_date is not None: conditions.append({ 'id' : condition_id, 'date' : condition_date })
    else:
        for condition_id, condition_dates in person2condition2date[person_id].items():
            for condition_date in condition_dates:
                if condition_date is not None: conditions.append({ 'id' : condition_id, 'date' : condition_date })
    
    # sort case events before control event if on the same day
    conditions.sort(key = lambda condition: condition['id'] in conditions_case)
    conditions.sort(key = lambda condition: condition['date'])
    return conditions

In [127]:
pwd

'/Users/owner/Downloads/faers_demo/assoc'

In [123]:
associations_file = open(output_file_name, 'w')

writer = csv.writer(associations_file, delimiter = ',')
writer.writerow(['age', 'gender', 'is_case', 'has_exposure', 'history'])

num_patients_noevents = 0
num_patients_case = 0
num_patients_control = 0
num_patients_self_control = 0
total_num_cases = 0
total_num_controls = 0

counts_control = []
counts_case = []

In [161]:
person2condition2date.keys()

dict_keys([2, 3, 4, 5, 7, 9, 10, 11, 12, 13, 17, 18, 20, 22, 23, 24, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 39, 41, 42, 44, 46, 47, 51, 56, 58, 59, 60, 61, 62, 64, 65, 66, 67, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 96, 97, 98, 99, 102, 105, 107, 108, 113, 115, 116, 118, 120, 122, 123, 124, 125, 127, 128, 129, 130, 133, 134, 137, 140, 142, 143, 144, 145, 150, 152, 154, 155, 156, 157, 162, 164, 165, 167, 168, 170, 171, 173, 174, 175, 176, 178, 180, 181, 182, 183, 184, 186, 187, 188, 190, 192, 195, 196, 201, 203, 206, 208, 209, 210, 211, 212, 213, 215, 216, 218, 219, 220, 221, 223, 224, 227, 229, 230, 235, 238, 239, 240, 243, 244, 245, 246, 247, 248, 251, 252, 255, 256, 257, 260, 263, 268, 269, 270, 271, 273, 274, 278, 279, 280, 281, 282, 283, 284, 285, 288, 289, 291, 292, 295, 297, 300, 303, 304, 308, 310, 311, 314, 316, 317, 318, 320, 321, 324, 325, 328, 331, 332, 336, 337, 342, 343, 345, 346, 347, 348, 349, 350, 352, 353, 356, 357, 

In [164]:
get_drugs(349)

[]

In [124]:
for person_id in tqdm.tqdm(person2condition2date.keys()):
    drugs = get_drugs(person_id)
    conditions = get_conditions(person_id)
    
    # advance drugs to modern era
    i_drug_case = 0
    i_drug_control = 0
    while i_drug_case < len(drugs) and drugs[i_drug_case]['date'] < min_drug_date: i_drug_case += 1
    i_drug_control = i_drug_case
    
    # advance conditions so they're at least after a drug, if any
    i_condition = 0
    if len(drugs) > 0 and i_drug_case < len(drugs):
        while i_condition < len(conditions) and conditions[i_condition]['date'] < drugs[i_drug_case]['date']:
            i_condition += 1
    
    # skip the person if there's nothing left
    if i_drug_case == len(drugs) or i_condition == len(conditions):
        num_patients_noevents += 1
        continue
    
    if person_id in person2demog:
        demo = person2demog[person_id]
    else:
        demo = { 'yob': 'NA', 'gender': 'NA' }
    
    person_is_case = person_id in case_patients
    before_case = True
    
    history = set()
    
    num_cases = 0
    num_controls = 0
    
    while i_condition < len(conditions):
        condition_id = conditions[i_condition]['id']
        condition_date = conditions[i_condition]['date']
        age_at_condition = 'NA' if demo['yob'] == 'NA' else condition_date.year - demo['yob']
        i_condition += 1
        
        skip_condition = \
          (condition_id in conditions_case and num_cases >= num_cases_per_patient) or \
          (condition_id not in conditions_case and (\
            num_controls >= num_controls_per_patient or \
            (person_is_case and (self_control_style == "none" or (self_control_style == "past" and not before_case)))))
        
        if skip_condition:
            continue
        
        drugs_associated = set()
        
        if condition_id in conditions_case:
            # advance drugs to be in case exposure window
            while i_drug_case < len(drugs) and condition_date - drugs[i_drug_case]['date'] > timedelta(days = exposure_window_case): i_drug_case += 1
            
            i_drug = i_drug_case
            # for every exposure in window, write an event
            # we 'sort' drugs before conditions by using >= 0 days difference
            while i_drug < len(drugs) and (condition_date - drugs[i_drug]['date']) >= timedelta(days = 0):
                if drugs[i_drug]['id'] not in drugs_associated and num_cases < num_cases_per_patient:
                    
                    writer.writerow([age_at_condition, demo['gender'], 1, \
                                     int(drugs[i_drug]['id'] in drugs_exposure), sort_and_str(history)])
                    drugs_associated.add(drugs[i_drug]['id'])
                    num_cases += 1
                i_drug += 1
            
            before_case = False
        else:
            while i_drug_control < len(drugs) and (condition_date - drugs[i_drug_control]['date']) > timedelta(days = exposure_window_control): i_drug_control += 1
            
            i_drug = i_drug_control
            while i_drug < len(drugs) and (condition_date - drugs[i_drug]['date']) >= timedelta(days = 0):
                if drugs[i_drug]['id'] not in drugs_associated and num_controls < num_controls_per_patient:
                    
                    writer.writerow([age_at_condition, demo['gender'], 0, \
                                     int(drugs[i_drug]['id'] in drugs_exposure), sort_and_str(history)])
                    drugs_associated.add(drugs[i_drug]['id'])
                    num_controls += 1
                i_drug += 1
        
        if condition_id in code2chapter:
            history.add(code2chapter[condition_id])
    
    if num_cases > 0:
        num_patients_case += 1
        if num_controls > 0:
            num_patients_self_control += 1
    if num_controls > 0:
        num_patients_control += 1
    
    total_num_cases += num_cases
    total_num_controls += num_controls
    
    if len(counts_case) < num_cases + 1:
        counts_case += [ 0 ] * (num_cases - len(counts_case) + 1)
    counts_case[num_cases] += 1
    if len(counts_control) < num_controls + 1:
        counts_control += [ 0 ] * (num_controls - len(counts_control) + 1)
    counts_control[num_controls] += 1


associations_file.close()

100%|██████████| 597/597 [00:00<00:00, 46071.75it/s]


In [126]:
print ("Processed patients: %d with no events, %d cases, %d controls, %d self-controls, %d total\naverage of %.2f events per case, %.2f events per control\n" % \
      ( num_patients_noevents, num_patients_case, num_patients_control, num_patients_self_control, len(person2condition2date), \
        float(total_num_cases) / float(num_patients_case), float(total_num_controls) / float(num_patients_control))
)
# add extra 0s at end
if len(counts_case) < len(counts_control):
    counts_case += [ 0 ] * (len(counts_control) - len(counts_case))

if len(counts_control) < len(counts_case):
    counts_control += [ 0 ] * (len(counts_case) - len(counts_control))

counts_file = open(counts_file_name, 'w')

writer = csv.writer(counts_file, delimiter = ',')
writer.writerow(['n', 'case', 'control'])

for i in range(len(counts_case)):
    writer.writerow([i, counts_case[i], counts_control[i]])

counts_file.close()



Processed patients: 529 with no events, 2 cases, 56 controls, 2 self-controls, 597 total
average of 1.00 events per case, 7.36 events per control



In [16]:
synpuf_person = pd.read_sql_query('''SELECT * FROM synpuf5.person;''', cnx)
#synpuf_person = synpuf_person.set_index('person_id')
synpuf_person

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,time_of_birth,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,1,8507,1923,5,1,None,8527,38003564,1,None,None,00013D2EFD8E45D1,1,None,1,None,1,None
1,2,8507,1943,1,1,None,8527,38003564,2,None,None,00016F745862898F,1,None,1,None,1,None
2,3,8532,1936,9,1,None,8527,38003564,3,None,None,0001FDD721E223DC,2,None,1,None,1,None
3,4,8507,1941,6,1,None,0,38003563,4,None,None,00021CA6FF03E670,1,None,5,None,5,None
4,5,8507,1936,8,1,None,8527,38003564,5,None,None,00024B3D2352D2D0,1,None,1,None,1,None
5,6,8507,1943,10,1,None,8516,38003564,6,None,None,0002DAE1C81CC70D,1,None,2,None,2,None
6,7,8507,1922,7,1,None,8527,38003564,7,None,None,0002F28CE057345B,1,None,1,None,1,None
7,8,8507,1935,9,1,None,8527,38003564,8,None,None,000308435E3E5B76,1,None,1,None,1,None
8,9,8532,1976,9,1,None,8527,38003564,9,None,None,000345A39D4157C9,2,None,1,None,1,None
9,10,8532,1938,10,1,None,8516,38003564,10,None,None,00036A21B65B0206,2,None,2,None,2,None


In [17]:
synpuf_person = pd.read_sql_query('''select person_id, drug_concept_id, drug_era_start_date
from drug_era;''', cnx)

ProgrammingError: (psycopg2.ProgrammingError) relation "drug_era" does not exist
LINE 2: from drug_era;
             ^
 [SQL: 'select person_id, drug_concept_id, drug_era_start_date\nfrom drug_era;'] (Background on this error at: http://sqlalche.me/e/f405)